In [26]:
# -*- coding: utf-8 -*-
"""Polynomial Anomoly Detector with GARCH method and raw error method
"""
# Author: Yinchen Wu <yinchen@uchicago.edu>

from __future__ import division
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
import random
from arch import arch_model
import pandas as pd
import math
import pmdarima as pm
from pmdarima import model_selection
import os
import dis
import statistics
from sklearn import metrics
import sklearn
from detectorA import DetectorA
from detectorB import DetectorB


import warnings
from collections import defaultdict

from ptsa.utils.utility import _get_sklearn_version

if _get_sklearn_version() > 20:
    from inspect import signature
else:
    from sklearn.externals.funcsigs import signature

import abc
import six

import numpy as np
from numpy import percentile
from scipy.special import erf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.utils import deprecated
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.multiclass import check_classification_targets
import scipy.stats

from sklearn_base import _pprint
from ptsa.utils.utility import precision_n_scores

In [2]:
data = pd.read_csv('A-1.txt', header = None).squeeze()

In [3]:
from __future__ import division
from __future__ import print_function

from sklearn.ensemble import IsolationForest
from sklearn.utils.validation import check_is_fitted
from sklearn.utils import check_array
from sklearn.utils import column_or_1d


import numpy as np
from numpy import percentile
import matplotlib.pyplot as plt
import random
from arch import arch_model
import pandas as pd
import math
import pmdarima as pm
from pmdarima import model_selection
import os
import dis
import statistics
from sklearn import metrics
from warnings import filterwarnings

class IForest(DetectorB):
    """The IsolationForest 'isolates' observations by randomly selecting a
    feature and then randomly selecting a split value between the maximum and
    minimum values of the selected feature.
    See :cite:`liu2008isolation,liu2012isolation` for details.
    IForest assumes a correlation between the distance of tree structure and the
    likelihood of being an anomly. Obviously, random partitioning produces noticeably 
    shorter paths for anomalies. Hence, by randomly creating tree structure on the 
    dataset, we may calculate the expected distance of each point and thus a measure
    of its normality.
    Parameters
    ----------
    n_estimators : int, optional (default=100)
        The number of base estimators in the ensemble.
    max_samples : int or float, optional (default="auto")
        The number of samples to draw from X to train each base estimator.
            - If int, then draw `max_samples` samples.
            - If float, then draw `max_samples * X.shape[0]` samples.
            - If "auto", then `max_samples=min(256, n_samples)`.
    contamination : float in (0., 0.5), optional (default=0.1)
        The amount of contamination of the data set, i.e. the proportion
        of outliers in the data set. Used when fitting to define the threshold
        on the decision function.
    max_features : int or float, optional (default=1.0)
        The number of features to draw from X to train each base estimator.
            - If int, then draw `max_features` features.
            - If float, then draw `max_features * X.shape[1]` features.
            - this attribute is useless of a sensor timeseries anomly
    bootstrap : bool, optional (default=False)
        If True, individual trees are fit on random subsets of the training
        data sampled with replacement. If False, sampling without replacement
        is performed.
    n_jobs : integer, optional (default=1)
        The number of jobs to run in parallel for both `fit` and `predict`.
        If -1, then the number of jobs is set to the number of cores.
    random_state : int, RandomState instance or None, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`.
    Attributes
    ----------
    estimators_ : list of DecisionTreeClassifier
        The collection of fitted sub-estimators.
    estimators_samples_ : list of arrays
        The subset of drawn samples (i.e., the in-bag samples) for each base
        estimator.
    max_samples_ : integer
        The actual number of samples
    decision_scores_ : numpy array of shape (n_samples,)
        The outlier scores of the training data.
        The higher, the more abnormal. Outliers tend to have higher
        scores. This value is available once the detector is
        fitted.
    threshold_ : float
        The threshold is based on ``contamination``. It is the
        ``n_samples * contamination`` most abnormal samples in
        ``decision_scores_``. The threshold is calculated for generating
        binary outlier labels.
    labels_ : int, either 0 or 1
        The binary labels of the training data. 0 stands for inliers
        and 1 for outliers/anomalies. It is generated by applying
        ``threshold_`` on ``decision_scores_``.
    """

    def __init__(self, n_estimators=100,
                 max_samples="auto",
                 contamination=0.1,
                 max_features=1.,
                 bootstrap=False,
                 n_jobs=1,
                 behaviour='old',
                 random_state=None,
                 verbose=0):
                
        self.contamination = contamination
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.max_features = max_features
        self.bootstrap = bootstrap
        self.n_jobs = n_jobs
        self.behaviour = behaviour
        self.random_state = random_state
        self.verbose = verbose

    def fit(self, X, y=None):
        """Fit detector. y is ignored in unsupervised methods.
        Parameters
        ----------
        X : numpy array of shape (n_samples, n_features)
            The input samples.
        y : Ignored
            Not used, present for API consistency by convention.
        Returns
        -------
        self : object
            Fitted estimator.
        """
        # validate inputs X and y (optional)
        try: 
            X = check_array(X)
        except:
            X = X.reshape(-1, 1)
        
        X = check_array(X)
            
        self._set_n_classes(y)

        # Check the sklearn version 
        sklearn_version = _get_sklearn_version()
        if sklearn_version == 21:
            self.detector_ = IsolationForest(n_estimators=self.n_estimators,
                                             max_samples=self.max_samples,
                                             contamination=self.contamination,
                                             max_features=self.max_features,
                                             bootstrap=self.bootstrap,
                                             n_jobs=self.n_jobs,
                                             behaviour=self.behaviour,
                                             random_state=self.random_state,
                                             verbose=self.verbose)

        # Do not pass behaviour argument when sklearn version is < 0.20 or >0.21
        else:  # pragma: no cover
            self.detector_ = IsolationForest(n_estimators=self.n_estimators,
                                             max_samples=self.max_samples,
                                             contamination=self.contamination,
                                             max_features=self.max_features,
                                             bootstrap=self.bootstrap,
                                             n_jobs=self.n_jobs,
                                             random_state=self.random_state,
                                             verbose=self.verbose)

        self.detector_.fit(X=X, y=None, sample_weight=None)

        # invert decision_scores_. Outliers comes with higher outlier scores.
        self.decision_scores_ = invert_order(
            self.detector_.decision_function(X))
        self._process_decision_scores()
        return self

    def decision_function(self, X):
        """Predict raw anomaly score of X using the fitted detector.
        The anomaly score of an input sample is computed based on different
        detector algorithms. For consistency, outliers are assigned with
        larger anomaly scores.
        Parameters
        ----------
        X : numpy array of shape (n_samples, n_features)
            The training input samples. Sparse matrices are accepted only
            if they are supported by the base estimator.
        Returns
        -------
        anomaly_scores : numpy array of shape (n_samples,)
            The anomaly score of the input samples.
        """
        check_is_fitted(self, ['decision_scores_', 'threshold_', 'labels_'])
        # invert outlier scores. Outliers comes with higher outlier scores
        return invert_order(self.detector_.decision_function(X))


    def estimators_(self):
        """The collection of fitted sub-estimators.
        Decorator for scikit-learn Isolation Forest attributes.
        """
        return self.detector_.estimators_


    def estimators_samples_(self):
        """The subset of drawn samples (i.e., the in-bag samples) for
        each base estimator.
        Decorator for scikit-learn Isolation Forest attributes.
        """
        return self.detector_.estimators_samples_

    def max_samples_(self):
        """The actual number of samples.
        Decorator for scikit-learn Isolation Forest attributes.
        """
        return self.detector_.max_samples_

    def _set_n_classes(self, y):
        """Set the number of classes if `y` is presented, which is not
        expected. It could be useful for multi-class outlier detection.
        Parameters
        ----------
        y : numpy array of shape (n_samples,)
            Ground truth.
        Returns
        -------
        self
        """

        self._classes = 2  # default as binary classification
        if y is not None:
            check_classification_targets(y)
            self._classes = len(np.unique(y))
            warnings.warn(
                "y should not be presented in unsupervised learning.")
        return self
    
    def _process_decision_scores(self):
        """Internal function to calculate key attributes:
        - threshold_: used to decide the binary label
        - labels_: binary labels of training data
        Returns
        -------
        self
        """

        self.threshold_ = percentile(self.decision_scores_,
                                     100 * (1 - self.contamination))
        self.labels_ = (self.decision_scores_ > self.threshold_).astype(
            'int').ravel()

        # calculate for predict_proba()

        self._mu = np.mean(self.decision_scores_)
        self._sigma = np.std(self.decision_scores_)

        return self

0            NaN
1            NaN
2            NaN
3       0.976049
4       0.976191
5       0.976298
6       0.976378
7       0.976436
8       0.976478
9       0.976505
10      0.976523
11      0.976534
12      0.976539
13      0.976541
14      0.976540
15      0.976537
16      0.976534
17      0.976530
18      0.976894
19      0.978159
20      0.976829
21      0.979305
22      0.981561
23      0.982510
24      0.982620
25      0.981345
26      0.979239
27      0.977119
28      0.974552
29      0.971976
          ...   
8350    0.964676
8351    0.964741
8352    0.965223
8353    0.966017
8354    0.967020
8355    0.968136
8356    0.969286
8357    0.970407
8358    0.971457
8359    0.972408
8360    0.973245
8361    0.973964
8362    0.974568
8363    0.975065
8364    0.975466
8365    0.975782
8366    0.976027
8367    0.976212
8368    0.976349
8369    0.976446
8370    0.976513
8371    0.976558
8372    0.976584
8373    0.976597
8374    0.976602
8375    0.976600
8376    0.976595
8377    0.9765

In [17]:
def convert(X, n):
    X = pd.Series(X)
    L = []
    for i in range(n):
        L.append(X.shift(i))
    df = pd.concat(L, axis = 1)
    return df

In [14]:
X = np.arange(100)

In [15]:
pd.Series(X).shift(1)

0      NaN
1      0.0
2      1.0
3      2.0
4      3.0
5      4.0
6      5.0
7      6.0
8      7.0
9      8.0
10     9.0
11    10.0
12    11.0
13    12.0
14    13.0
15    14.0
16    15.0
17    16.0
18    17.0
19    18.0
20    19.0
21    20.0
22    21.0
23    22.0
24    23.0
25    24.0
26    25.0
27    26.0
28    27.0
29    28.0
      ... 
70    69.0
71    70.0
72    71.0
73    72.0
74    73.0
75    74.0
76    75.0
77    76.0
78    77.0
79    78.0
80    79.0
81    80.0
82    81.0
83    82.0
84    83.0
85    84.0
86    85.0
87    86.0
88    87.0
89    88.0
90    89.0
91    90.0
92    91.0
93    92.0
94    93.0
95    94.0
96    95.0
97    96.0
98    97.0
99    98.0
Length: 100, dtype: float64

In [23]:
M = convert(data,20)

In [24]:
M

,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0.976049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.976191,0.976049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.976298,0.976191,0.976049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.976378,0.976298,0.976191,0.976049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.976436,0.976378,0.976298,0.976191,0.976049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.976478,0.976436,0.976378,0.976298,0.976191,0.976049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.976505,0.976478,0.976436,0.976378,0.976298,0.976191,0.976049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.976523,0.976505,0.976478,0.976436,0.976378,0.976298,0.976191,0.976049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.976534,0.976523,0.976505,0.976478,0.976436,0.976378,0.976298,0.976191,0.976049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.976539,0.976534,0.976523,0.976505,0.976478,0.976436,0.976378,0.976298,0.976191,0.976049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
convert(data, 1)

,0
0,0.976049
1,0.976191
2,0.976298
3,0.976378
4,0.976436
5,0.976478
6,0.976505
7,0.976523
8,0.976534
9,0.976539
